In [6]:
import struct

def read_thermo_lib(file_path):
    species_data = []
    with open(file_path, 'rb') as f:
        # Adjust endianness and record marker size based on your system
        endian = '<'  # '<' for little-endian, '>' for big-endian
        record_marker_fmt = endian + 'I'  # 'I' for 4-byte unsigned int
        record_marker_size = struct.calcsize(record_marker_fmt)

        # Helper function to read Fortran unformatted records
        def read_record():
            # Read the record marker
            marker = f.read(record_marker_size)
            if not marker:
                return None  # End of file
            record_size = struct.unpack(record_marker_fmt, marker)[0]
            # Read the record data
            record_data = f.read(record_size)
            # Read the closing record marker
            closing_marker = f.read(record_marker_size)
            return record_data

        # Read the header record
        header_data = read_record()
        if header_data is None:
            raise EOFError("File is empty or header is missing.")

        # Unpack the header
        header_fmt = endian + '4d 3i 10s'
        header_size = struct.calcsize(header_fmt)
        if len(header_data) != header_size:
            raise ValueError(f"Header size mismatch: expected {header_size}, got {len(header_data)}")

        tgl = struct.unpack(endian + '4d', header_data[0:32])
        ngl, ns, nall = struct.unpack(endian + '3i', header_data[32:44])
        Thdate = struct.unpack(endian + '10s', header_data[44:54])[0].decode('ascii').strip()

        print(f"Header: tgl={tgl}, ngl={ngl}, ns={ns}, nall={nall}, Thdate={Thdate}")

        # Read species records
        for i in range(nall):
            print(i)

            record_data = read_record()
            if record_data is None:
                break  # End of file

            offset = 0

            # Unpack species common data
            species_fmt = endian + '15s i 6s'
            species_size = struct.calcsize(species_fmt)
            name, ntl, date = struct.unpack(species_fmt, record_data[offset:offset+species_size])
            name = name.decode('ascii').strip()
            date = date.decode('ascii').strip()
            offset += species_size

            # Unpack sym(j) and fno(j)
            sym_fno = []
            for _ in range(5):
                sym_fmt = endian + '2s d'  # sym(j), fno(j)
                sym_size = struct.calcsize(sym_fmt)
                sym_j, fno_j = struct.unpack(sym_fmt, record_data[offset:offset+sym_size])
                sym_j = sym_j.decode('ascii').strip()
                sym_fno.append((sym_j, fno_j))
                offset += sym_size

            # Unpack ifaz
            ifaz_fmt = endian + 'i'
            ifaz_size = struct.calcsize(ifaz_fmt)
            ifaz = struct.unpack(ifaz_fmt, record_data[offset:offset+ifaz_size])[0]
            offset += ifaz_size

            # Unpack tl(1:2)
            tl_fmt = endian + '2d'
            tl_size = struct.calcsize(tl_fmt)
            tl = struct.unpack(tl_fmt, record_data[offset:offset+tl_size])
            offset += tl_size

            # Unpack mwt
            mwt_fmt = endian + 'd'
            mwt_size = struct.calcsize(mwt_fmt)
            mwt = struct.unpack(mwt_fmt, record_data[offset:offset+mwt_size])[0]
            offset += mwt_size

            # Initialize thermo_coeffs
            thermo_coeffs = []

            # Check if ntl > 0 to read thermo coefficients
            if ntl > 0:
                # Number of coefficients is 9 coefficients per temperature interval
                num_coeffs = 9 * ntl
                thermo_fmt = endian + f'{num_coeffs}d'
                thermo_size = struct.calcsize(thermo_fmt)

                
                # TODO: REMOVE FROM DEBUGGIN
                if i == 1270:
                    print(record_data[offset:offset+thermo_size])
                    continue

                try:
                    thermo_data = struct.unpack(thermo_fmt, record_data[offset:offset+thermo_size])
                except:
                    print(thermo_size)
                    print(offset)
                    print(offset+thermo_size)
                    print(len(record_data[offset:offset+thermo_size]))
                    thermo_data = struct.unpack(thermo_fmt, record_data[offset:offset+144])


                offset += thermo_size

                # Organize coefficients by temperature interval
                for i in range(ntl):
                    coeffs = thermo_data[i*9:(i+1)*9]
                    thermo_coeffs.append(coeffs)
            else:
                # For ntl == 0, only thermo(1,1) is written (formation enthalpy)
                thermo_fmt = endian + 'd'
                thermo_size = struct.calcsize(thermo_fmt)
                thermo_data = struct.unpack(thermo_fmt, record_data[offset:offset+thermo_size])
                offset += thermo_size
                thermo_coeffs.append(thermo_data)

            species = {
                'name': name,
                'ntl': ntl,
                'date': date,
                'elements': sym_fno,
                'ifaz': ifaz,
                'tl': tl,
                'mwt': mwt,
                'thermo_coeffs': thermo_coeffs
            }
            species_data.append(species)

        return species_data

# Example usage
if __name__ == '__main__':
    file_path = 'thermo.lib'  # Replace with your actual file path


    data = read_thermo_lib(file_path)


    # Print data for each species
    for species in data:
        print(f"Species Name: {species['name']}")
        print(f"Number of Temperature Intervals: {species['ntl']}")
        print(f"Date: {species['date']}")
        print(f"Elements:")
        for sym, fno in species['elements']:
            print(f"  Symbol: {sym}, Number: {fno}")
        print(f"Phase Indicator (ifaz): {species['ifaz']}")
        print(f"Temperature Limits (tl): {species['tl']}")
        print(f"Molecular Weight (mwt): {species['mwt']}")
        if species['ntl'] > 0:
            print(f"Thermo Coefficients:")
            for idx, coeffs in enumerate(species['thermo_coeffs'], start=1):
                print(f"  Interval {idx}: {coeffs}")
        print("\n")


Header: tgl=(200.0, 1000.0, 6000.0, 20000.0), ngl=1, ns=4, nall=4, Thdate=9/09/04
0
1
144
103
247
72


error: unpack requires a buffer of 144 bytes

In [ ]:
import struct

def read_thermo_lib(file_path):
    species_data = []
    with open(file_path, 'rb') as f:
        # Try different combinations of endianness and record marker size
        for endian in ['<', '>']:
            for marker_fmt in ['I', 'Q']:  # 'I' is 4 bytes, 'Q' is 8 bytes
                f.seek(0)  # Reset file pointer
                record_marker_fmt = endian + marker_fmt
                record_marker_size = struct.calcsize(record_marker_fmt)
                print(f"Trying endianness '{endian}' and record marker format '{marker_fmt}'")
                try:
                    # Read header record
                    marker = f.read(record_marker_size)
                    if not marker:
                        raise EOFError("File is empty or header is missing.")
                    record_size = struct.unpack(record_marker_fmt, marker)[0]
                    header_data = f.read(record_size)
                    closing_marker = f.read(record_marker_size)
                    # Unpack header
                    header_fmt = endian + '4d 3i 10s'
                    header_size = struct.calcsize(header_fmt)
                    if len(header_data) != header_size:
                        print(f"Header size mismatch: expected {header_size}, got {len(header_data)}")
                        continue
                    tgl = struct.unpack(endian + '4d', header_data[0:32])
                    ngl, ns, nall = struct.unpack(endian + '3i', header_data[32:44])
                    Thdate = struct.unpack(endian + '10s', header_data[44:54])[0].decode('ascii').strip()
                    print(f"Header read successfully with endianness '{endian}' and marker format '{marker_fmt}'")
                    print(f"tgl={tgl}, ngl={ngl}, ns={ns}, nall={nall}, Thdate={Thdate}")
                    # Proceed to read species data
                    for _ in range(nall):
                        # Read species record
                        marker = f.read(record_marker_size)
                        if not marker:
                            raise EOFError("Unexpected end of file while reading species records.")
                        record_size = struct.unpack(record_marker_fmt, marker)[0]
                        record_data = f.read(record_size)
                        closing_marker = f.read(record_marker_size)
                        offset = 0
                        # Unpack species common data
                        species_fmt = endian + '15s i 6s'
                        species_size = struct.calcsize(species_fmt)
                        name, ntl, date = struct.unpack(species_fmt, record_data[offset:offset+species_size])
                        name = name.decode('ascii').strip()
                        date = date.decode('ascii').strip()
                        offset += species_size
                        # Unpack sym(j) and fno(j)
                        sym_fno = []
                        for _ in range(5):
                            sym_fmt = endian + '2s d'
                            sym_size = struct.calcsize(sym_fmt)
                            sym_j, fno_j = struct.unpack(sym_fmt, record_data[offset:offset+sym_size])
                            sym_j = sym_j.decode('ascii').strip()
                            sym_fno.append((sym_j, fno_j))
                            offset += sym_size
                        # Unpack ifaz
                        ifaz_fmt = endian + 'i'
                        ifaz_size = struct.calcsize(ifaz_fmt)
                        ifaz = struct.unpack(ifaz_fmt, record_data[offset:offset+ifaz_size])[0]
                        offset += ifaz_size
                        # Unpack tl(1:2)
                        tl_fmt = endian + '2d'
                        tl_size = struct.calcsize(tl_fmt)
                        tl = struct.unpack(tl_fmt, record_data[offset:offset+tl_size])
                        offset += tl_size
                        # Unpack mwt
                        mwt_fmt = endian + 'd'
                        mwt_size = struct.calcsize(mwt_fmt)
                        mwt = struct.unpack(mwt_fmt, record_data[offset:offset+mwt_size])[0]
                        offset += mwt_size
                        # Initialize thermo_coeffs
                        thermo_coeffs = []
                        # Check if ntl > 0 to read thermo coefficients
                        if ntl > 0:
                            num_coeffs = 9 * ntl
                            thermo_fmt = endian + f'{num_coeffs}d'
                            thermo_size = struct.calcsize(thermo_fmt)
                            thermo_data = struct.unpack(thermo_fmt, record_data[offset:offset+thermo_size])
                            offset += thermo_size
                            # Organize coefficients by temperature interval
                            for i in range(ntl):
                                coeffs = thermo_data[i*9:(i+1)*9]
                                thermo_coeffs.append(coeffs)
                        else:
                            # For ntl == 0, only thermo(1,1) is written (formation enthalpy)
                            thermo_fmt = endian + 'd'
                            thermo_size = struct.calcsize(thermo_fmt)
                            thermo_data = struct.unpack(thermo_fmt, record_data[offset:offset+thermo_size])
                            offset += thermo_size
                            thermo_coeffs.append(thermo_data)
                        species = {
                            'name': name,
                            'ntl': ntl,
                            'date': date,
                            'elements': sym_fno,
                            'ifaz': ifaz,
                            'tl': tl,
                            'mwt': mwt,
                            'thermo_coeffs': thermo_coeffs
                        }
                        species_data.append(species)
                    # If we reach this point without exceptions, return the data
                    return species_data
                except Exception as e:
                    print(f"Failed with endianness '{endian}' and marker format '{marker_fmt}': {e}")
                    species_data = []
                    f.seek(0)  # Reset file pointer
        raise ValueError("Failed to read thermo.lib with any combination of endianness and record marker size.")

# Example usage
if __name__ == '__main__':
    file_path = 'thermo.lib'  # Replace with your actual file path
    data = read_thermo_lib(file_path)
    # Print data for each species
    for species in data:
        print(f"Species Name: {species['name']}")
        print(f"Number of Temperature Intervals: {species['ntl']}")
        print(f"Date: {species['date']}")
        print(f"Elements:")
        for sym, fno in species['elements']:
            print(f"  Symbol: {sym}, Number: {fno}")
        print(f"Phase Indicator (ifaz): {species['ifaz']}")
        print(f"Temperature Limits (tl): {species['tl']}")
        print(f"Molecular Weight (mwt): {species['mwt']}")
        if species['ntl'] > 0:
            print(f"Thermo Coefficients:")
            for idx, coeffs in enumerate(species['thermo_coeffs'], start=1):
                print(f"  Interval {idx}: {coeffs}")
        print("\n")


Trying endianness '<' and record marker format 'I'
Header read successfully with endianness '<' and marker format 'I'
tgl=(200.0, 1000.0, 6000.0, 20000.0), ngl=1262, ns=2253, nall=2321, Thdate=9/09/04
Failed with endianness '<' and marker format 'I': unpack requires a buffer of 144 bytes
Trying endianness '<' and record marker format 'Q'
Header read successfully with endianness '<' and marker format 'Q'
tgl=(5.33898752e-315, 5.35137252e-315, 5.364384864e-315, 2.6784960363345e-311), ngl=2253, ns=2321, nall=958406688, Thdate=/09/046   
Failed with endianness '<' and marker format 'Q': 
Trying endianness '>' and record marker format 'I'
Header size mismatch: expected 54, got 607777
Trying endianness '>' and record marker format 'Q'
Failed with endianness '>' and marker format 'Q': 


ValueError: Failed to read thermo.lib with any combination of endianness and record marker size.

In [ ]:
import numpy as np